In [1]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity

In [2]:
import touschek_pack.functions as tousfunc

In [3]:
import touschek_pack.Classes as tousclass

In [45]:
from IPython.display import display, Latex

In [4]:
# eu preciso lembrar de depois acrescentar uma função que faça o trakcing nao tornando explicito a diferenciação entre desvios de energia 
# positivos e negativos para contabilizar de maneira correta a taxa de espalhamento touschek porque na biblioteca lifetime a taxa de espalhamento
# calculada esta implementada com base nos espalhamentos tanto negativos como positivos

In [5]:
# This function will probably will be in my repositories
# I dont know if I will use it again, but it seems to me that it could be uselfull in some point

def extract_delt(groups, deltas):
    c = 0
    big_list = []
    for lists in groups:
        lil_list = []
        for _ in lists:
            lil_list.append(c)
            c+=1
            
        big_list.append(lil_list)
    
    sep_deltas = []
    comp_l = []

    for iten in big_list:
        sep_deltas.append(deltas[iten])
        comp_l.append(len(iten))


    return sep_deltas, comp_l

In [6]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)

In [7]:
# lembre-se que queremos contruir uma função que transforme dados que eu já possuo em tabelas
# uma biblioteca bastante util para isso seria a bib pandas
# vamos tentar usar ela para nossos calculos


In [226]:
spos = pyaccel.lattice.find_spos(acc, indices='closed')
s = np.round([spos[1046]], 2)

ress = tousan.fast_aquisition(s, 'pos')
res, fp, dp = ress

# para este caso específico a posição analisada é spos[1046], ou seja, o indice em que a simulação tracking começa é em 1046
# a para este valor teremos então diversas posições em que o eletron é perdido e a partir disso eu vou ter organizar meu pensamento

lostinds, deltas = np.zeros(len(res)), np.zeros(len(res))
for idx,iten in enumerate(res):
    tloss, ellost, delta = iten
    lostinds[idx] = ellost
    deltas[idx] = delta # alguns elétrons possuem desvio de energia abaixo da aceitancia e acabam não sendo perdidos

lostinds = np.intp(lostinds)

# deltas são os desvios de energia perdidos pelo tracking
lost_positions = spos[lostinds]
lost_positions = np.round(lost_positions, 2)

In [268]:
data = pd.DataFrame({'lost_pos_by_tracking': lost_positions}) # create the dataframe that is obtained by tracking


In [269]:
lost_pos_column = (data.groupby('lost_pos_by_tracking').groups).keys()
data = pd.DataFrame({'lost_pos_by_tracking':lost_pos_column}) # this step agroups the lost_positions

In [270]:

fact = 0.03
step = int((deltas[0]+deltas[-1])/fact)
itv_track = np.linspace(deltas[0], deltas[-1], step) # method learned by fac repositories

# scat_lost_df = pd.DataFrame(f'{s}':) # dataframe will contain the scattered positions and the lost positions after scattering

itv_delta = []
for current, next_iten in zip(itv_track, itv_track[1:]):
    data['{:.2f} % < delta < {:.2f} %'.format(current*1e2, next_iten*1e2)] = np.zeros(len(list(lost_pos_column))) # this step creates new columns in the dataframe and fill with zeros
    itv_delta.append((current, next_iten))
    # Next step must calculate each matrix element from the dataframe

var = list(data.index) 
# quando as duas variáveis são iguais isso acab resultando em um erro então estou colocando essa condição.
if var == lost_pos_column:
    pass
else:
    data = data.set_index('lost_pos_by_tracking')


for idx, lost_pos in enumerate(lost_positions): # essas duas estruturas de repetição são responsáveis por calcular 
    # o percentual dos eletrons que possuem um determinado desvio de energia e se perdem em um intervalo de desvio de energia específico
    delta = deltas[idx]
    lps = []
    for j, interval in enumerate(itv_delta):
        if j == 0:
            if interval[0]<= delta <= interval[1]:
                data.loc[lost_pos, '{:.2f} % < delta < {:.2f} %'.format(interval[0]*1e2, interval[1]*1e2)] += 1
        else:
            if interval[0]< delta <= interval[1]:
                data.loc[lost_pos, '{:.2f} % < delta < {:.2f} %'.format(interval[0]*1e2, interval[1]*1e2)] += 1

data = data / len(deltas)

In [237]:
data

,2.91 % < delta < 5.38 %,5.38 % < delta < 7.86 %,7.86 % < delta < 10.33 %,10.33 % < delta < 12.80 %,12.80 % < delta < 15.27 %
lost_pos_by_tracking,,,,,
88.01,0.000000,0.000000,0.000000,0.000000,0.148148
88.16,0.000000,0.000000,0.000000,0.129630,0.052469
88.33,0.000000,0.000000,0.027778,0.070988,0.000000
88.53,0.000000,0.000000,0.040123,0.000000,0.000000
88.76,0.000000,0.000000,0.006173,0.000000,0.000000
95.46,0.000000,0.000000,0.083333,0.000000,0.000000
95.61,0.000000,0.055556,0.040123,0.000000,0.000000
95.78,0.000000,0.040123,0.000000,0.000000,0.000000
99.44,0.003086,0.000000,0.000000,0.000000,0.000000


In [362]:
dic = {}
for coluna, serie_booleana in data.eq(0).items():
    dic[coluna] = []
    val = dic[coluna]
    for indice, valor_booleano in serie_booleana.items():
        if valor_booleano == False:
            val.append(indice)


In [370]:
for iten in dic:
    if iten in 

{'2.91 % < delta < 5.38 %': [99.44,
  99.53,
  151.36,
  173.22,
  173.37,
  173.54,
  176.74,
  176.89,
  177.04,
  203.2,
  203.4,
  510.32,
  510.49,
  513.85,
  514.0,
  514.07,
  514.16,
  514.24],
 '5.38 % < delta < 7.86 %': [95.61, 95.78, 107.48, 151.12, 151.36],
 '7.86 % < delta < 10.33 %': [88.33, 88.53, 88.76, 95.46, 95.61],
 '10.33 % < delta < 12.80 %': [88.16, 88.33],
 '12.80 % < delta < 15.27 %': [88.01, 88.16]}

In [ ]:
ndf = scat_lost_df.set_index(lost_positions)

ndf = ndf[~ndf.index.duplicated(keep='first')]

dataf = ndf.transpose()

In [151]:
dataf.columns

[84.29923250000009]    185
Name: 88.33, dtype: int64

In [ ]:
if data.values.sum() == 1:
    pass
else:
    data = data/len(deltas)

In [21]:
ltime = pyaccel.lifetime.Lifetime(acc)
tous_rate = ltime.touschek_data['rate']
spos = pyaccel.lattice.find_spos(acc, indices='closed')

npt = int((spos[-1]-spos[0])/0.1)


scalc = np.linspace(spos[0], spos[-1], npt)
rate_nom_lattice = np.interp(spos, scalc, tous_rate)


In [48]:
df_ltime = pd.DataFrame({'rate':rate_nom_lattice})


In [49]:
ind = np.arange(len(df_ltime))
df_index = spos[ind]

def_df_index = np.round(df_index, 2)
def_df_index

array([  0.  ,   0.  ,   0.  , ..., 517.89, 518.39, 518.39])

In [52]:
ndf = df_ltime.set_index(def_df_index)

In [58]:
len(ndf), len(ndf.drop_duplicates())

(6550, 5104)

In [63]:
plt.figure()
plt.plot(spos, rate_nom_lattice*1e7)

In [ ]:
accep = pyaccel.optics.calc_touschek_energy_acceptance(acc)

scalc, daccp, daccn = tousfunc.get_scaccep(acc, accep)

In [ ]:
b1 = ltime.touschek_data['touschek_coeffs']['b1']
b2 = ltime.touschek_data['touschek_coeffs']['b2']



In [ ]:
touschek_scattering_rate = []
# for idx,iten in enumerate(daccp):
#     touschek_scattering_rate.append(ltime.f_integral_simps(iten, b1[idx], b2[idx]))


In [ ]:
test_acceptance = np.array([daccp[1],daccp[2], daccp[3]])



In [ ]:
def f_function_arg(kappa, kappam, b1_, b2_):

    tau = (np.tan(kappa)**2)[:, None]
    taum = (np.tan(kappam)**2)[None, :]
    ratio = tau/taum/(1+tau)
    return ratio, tau, taum


In [ ]:
import scipy.special as special

In [ ]:
def f_function_arg_or(kappa, kappam, b1_, b2_):

    tau = (np.tan(kappa)**2)[:, None]
    taum = (np.tan(kappam)**2)[None, :]
    ratio = tau/taum/(1+tau)
    arg = (2*tau+1)**2 * (ratio - 1)/tau
    arg += tau - np.sqrt(tau*taum*(1+tau))
    arg -= (2+1/(2*tau))*np.log(ratio)
    arg *= np.sqrt(1+tau)
    bessel = np.exp(-(b1_-b2_)*tau)*special.i0e(b2_*tau)
    return arg * bessel

In [ ]:
npts = 300
kappa = np.linspace(daccp, np.pi/2, npts+1)

var = f_function_arg_or(kappa, daccp,b1,b2)


In [ ]:
tau.squeeze().transpose()

In [ ]:
matrix = np.array([[1,2,3],
                   [0,2,3],
                   [1,2,3]])
matrix.shape

matrix[1,0]



In [ ]:
tau.squeeze().shape